<a href="https://colab.research.google.com/github/castrokelly/eEDB-001-2023-4/blob/main/ensaio05_Modelos_Preditivos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Emissão de CO2

 A problemática da emissão de dióxido de carbono (CO2) por veículos automotores é um tema de grande relevância nos dias atuais, dada a crescente preocupação com as mudanças climáticas e a necessidade de reduzir as emissões de gases de efeito estufa.

A queima de combustíveis fósseis, como gasolina e diesel, nos motores de veículos é uma das principais fontes de emissão de CO2. Essas emissões contribuem significativamente para o aumento da concentração de gases de efeito estufa na atmosfera acelerando as mudanças climáticas. Além do CO2, outros poluentes atmosféricos provenientes dos escapamentos dos veículos podem ter impactos negativos na qualidade do ar e na saúde humana.

As políticas governamentais desempenham um papel crucial na condução da transição para um transporte mais sustentável. Incentivos fiscais, normas de eficiência veicular e metas ambiciosas de redução de emissões são estratégias que podem acelerar a mudança para um sistema de transporte menos poluente.

A aplicação de modelos preditivos baseados em Aprendizado de Máquina (AM) na previsão da emissão de CO2 por veículos automotores oferece uma série de contribuições no contexto da busca por soluções  para a redução dessas emissões. Incluindo:

1. **Otimização da Eficiência Veicular:**
   - Modelos preditivos podem analisar grandes conjuntos de dados, considerando variáveis como condições de tráfego, características do veículo, e padrões de condução. Essa análise permite identificar padrões complexos contribuindo para o desenvolvimento de estratégias para otimizar a eficiência dos veículos, reduzindo assim as emissões de CO2.

2. **Personalização de Recomendações:**
   - Modelos preditivos podem ser empregados para criar recomendações personalizadas aos motoristas. Por exemplo, um sistema baseado em AM pode fornecer sugestões específicas para manutenção adequada do veículo ou até mesmo recomendar a troca para veículos mais sustentáveis.


3. **Apoio a Políticas Públicas:**
   - Modelos preditivos podem oferecer insights  para os formuladores de políticas, permitindo a avaliação do impacto de diferentes medidas regulatórias na redução das emissões. Isso pode incluir a previsão de como padrões de emissão mais rigorosos ou incentivos para veículos de baixa emissão podem influenciar o cenário global de emissões de CO2.

4. **Monitoramento Contínuo:**
   - Modelos de Aprendizado de Máquina podem ser implementados para monitorar continuamente as emissões veiculares em tempo real. Isso possibilita uma resposta rápida a mudanças nas condições de tráfego, permitindo ajustes dinâmicos para maximizar a eficiência e minimizar as emissões.


Este é MVP para regressão da emissão de CO2 a partir de características construtivas, modelos e desempenho de veículos automotivos. Os dados estão disponibilizados na plataforma Kaggle: https://www.kaggle.com/datasets/rinichristy/2022-fuel-consumption-ratings

#Aprendizado de Máquina Supervisionado:Regressão

No aprendizado supervisionado cada instância é caracterizada por seus atributos e associada a um rótulo. O objetivo é treinar um modelo preditivo capaz de identificar uma função que mapeie os valores dos atributos preditivos para o valor da variável explicada (dependente). Esse modelo pode então ser empregado para prever valores de instâncias não vistas durante o treinamento. Este paradigma é fundamental quando se busca realizar tarefas de regressão, onde o objetivo é inferir o valor associado a uma nova entrada com base no aprendizado a partir de exemplos rotulados.

In [48]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

## Bibliotecas


In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
pip install scikit-learn --upgrade

In [ ]:
pip uninstall scikit-learn
pip install scikit-learn

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import TruncatedSVD, PCA

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score

from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix,
                             roc_curve,
                             roc_auc_score)
from imblearn.over_sampling import SMOTE
from tabulate import tabulate
warnings.filterwarnings('ignore')
sns.set_style('darkgrid')


ImportError: ignored

## Carregamento dos dados


In [ ]:
df = pd.read_csv('/content/dsEnsaio05.csv', encoding='utf-8', sep=',')

## Detalhes do dataframe


In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.sample(10)

In [ ]:
df.columns

>> Faltantes

In [ ]:
df.isna().sum()

>> Tamanho do data frame (dataset)

In [ ]:
df.shape

In [ ]:
print("tamanho:",df.size)

In [ ]:
X=df.drop(columns=["CO2 Emissions(g/km)", "Model Year"])
y=df["CO2 Emissions(g/km)"]

In [ ]:
X.head()

In [ ]:
numeric_columns=X.select_dtypes(exclude='object').columns
print(numeric_columns)

In [ ]:
categorical_columns = X.select_dtypes(include='object').columns
print(categorical_columns)

## 2. Correlação entre Atributos Preditivos Numéricos


In [ ]:
corr = X[numeric_columns].corr()

plt.figure(figsize=(15, 10))
plot = sns.heatmap(data=corr,
                   annot=True, annot_kws={'size': 10, 'fontweight': 'bold'},
                   cbar_kws={'extendfrac': .1, 'drawedges': True},
                   cmap='seismic', vmin=-1, linewidths=.2,
                   xticklabels=numeric_columns,
                   yticklabels=numeric_columns
                   )
plot.set_yticklabels(plot.get_yticklabels(), rotation=0)
plot.set_xticklabels(plot.get_xticklabels(), rotation=90)
plot.set_title('Correlação entre as variáveis\n', loc='center', fontsize=8)
plot.tick_params(labelsize=8)
plot.xaxis.tick_top()
plt.show()

## Pipelines de Pre-processamento

Os pipelines desempenham um papel importante em aplicações de Aprendizado de Máquina (AM) ao permitirem uma organização clara e reproduzível de processos complexos sequenciais. O Scikit-Learn oferece uma implementação robusta de pipelines, simplificando o desenvolvimento de modelos e facilitando a manutenção do código.

Funcionamento do Pipeline:

O pipeline é uma sequência de passos encadeados, cada um representando uma operação específica, como pré-processamento, transformação de dados oua própria construção do modelo de aprendizagem. A principal vantagem é que esses passos são encapsulados em uma única estrutura, tornando o código mais limpo, modular e  reproduzível.

Construção de Pipelines:

A construção de um pipeline envolve a definição de transformadores e estimadores, que são organizados em uma lista de tuplas. Cada tupla contém um nome identificador e a instância do transformador ou estimador. O pipeline é então criado usando a classe Pipeline, que aceita essa lista como argumento.

Os pipelines também são essenciais para a automatização de processos, facilitando a seleção de modelos, a otimização de hiperparâmetros e a manutenção do código em cenários complexos.

>>SimpleImputer(strategy='median'):

Esta etapa trata valores ausentes em features numéricas, substituindo-os pela mediana da respectiva coluna.

>>StandardScaler(with_mean=False):

Em seguida, os valores são escalados usando o StandardScaler para garantir que as features numéricas tenham média zero e variância unitária. O parâmetro with_mean=False indica que a média não deve ser centrada durante o escalonamento.

In [ ]:
numeric_feature = Pipeline(steps=[('handlingmissing',SimpleImputer(strategy='median')),('scaling',StandardScaler(with_mean=False))])
print(numeric_feature)

>>SimpleImputer(strategy='most_frequent'):

Assim como nas features numéricas, isso trata valores ausentes em features categóricas, substituindo-os pelo valor mais frequente da respectiva coluna.

>>OneHotEncoder():

Em seguida, aplica-se a codificação one-hot às features categóricas, convertendo-as em representações binárias.

>>StandardScaler(with_mean=False):

Similar às features numéricas, as features categóricas são escalonadas para garantir uma distribuição padrão. O parâmetro with_mean=False é usado aqui também.

In [ ]:
categorical_feature=Pipeline(steps=[('handlingmissing',SimpleImputer(strategy='most_frequent')),('encoding',OneHotEncoder()),('scaling',StandardScaler(with_mean=False))])
print(categorical_feature)

>> ColumnTransformer:

Aplica os pipelines acima. Ele usa os pipelines numeric_feature e categorical_feature para processar as colunas numéricas e categóricas, respectivamente.

In [ ]:
processing = ColumnTransformer([('numeric',numeric_feature,numeric_columns),
                                ('cat',categorical_feature,categorical_columns)])
processing

In [ ]:
processing.fit(X)
processed_X = processing.transform(X)

In [ ]:
print(X.shape)

In [ ]:
print(processed_X.shape)

In [ ]:
type(processed_X)

###Matrizes Esparças

scipy.sparse.csr_matrix é uma classe dentro do módulo scipy.sparse que representa uma matriz esparsa no formato CSR (Compressed Sparse Row). Matrizes esparsas são estruturas de dados eficientes para lidar com estruturas de dados que possuem a maioria dos elementos iguais a zero.

A matriz CSR armazena apenas os elementos não nulos, economizando espaço em comparação com uma matriz densa. Ela mantém três arrays principais:

data: Um array contendo os valores não nulos da matriz, armazenados em ordem de linha principal (row-major order).
indices: Um array de índices que indica a coluna correspondente a cada elemento em data.
indptr: Um array de ponteiros de índice, indicando onde começa e termina cada linha na matriz data e indices.

In [ ]:
print("Matriz Esparsa CSR:")
print(processed_X.toarray())

## Criando (Pipelines) os Modelos Preditivos

Construi-se  Pipelines de machine learning com pré-processamento, redução de dimensionalidade e correlação usando PCA (Principal Component Analysis) e, finalmente, treinando o modelo.

>> PCA

O `TruncatedSVD` (Singular Value Decomposition) é uma técnica de redução de dimensionalidade e correlação, que é frequentemente usada em dados esparsos. No código, utiliza-se `TruncatedSVD` com `n_components=800`, o que significa que se está mantendo 800 componentes principais após a redução de dimensionalidade.

Parâmetros:

- **`n_components`:** Especifica o número de componentes principais a serem mantidos após a redução de dimensionalidade.

- **`random_state`:** Controla a semente de geração de números aleatórios para garantir reprodutibilidade. Quando `random_state` é definido para um valor específico (por exemplo, `random_state=0`), os resultados do modelo serão os mesmos em diferentes execuções, desde que as condições de entrada também sejam as mesmas.

In [ ]:
from sklearn.neural_network import MLPRegressor
model0 = Pipeline(steps=[
    ('processing', processing),
    ("pca", TruncatedSVD(n_components=800, random_state=0)),
    ('modeling', MLPRegressor(hidden_layer_sizes=(100,), random_state=0))
])
model0.fit(X,y)
print(model0.score(X,y))

In [ ]:
# Diferentes configurações para hidden_layer_sizes
hidden_layer_sizes_list = [(100,), (50, 25), (100, 50, 25)]

# Iterando sobre as diferentes configurações
for hidden_layer_sizes in hidden_layer_sizes_list:
    # Criando o pipeline com pré-processamento, redução de dimensionalidade e modelo MLP
    model = Pipeline(steps=[
        ('processing', processing),
        ("pca", TruncatedSVD(n_components=800, random_state=0)),
        ('modeling', MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, random_state=0))
    ])
    print(f"Hidden Layer Sizes: {hidden_layer_sizes}")
    model.fit(X,y)
    print('Score: ', model0.score(X,y))
    print("\n"
    )


In [ ]:
# Calcule a proporção acumulativa da variância explicada
explained_variance_ratio_cumulative = np.cumsum(model0.named_steps['pca'].explained_variance_ratio_)

# Visualize a proporção acumulativa da variância explicada
plt.plot(explained_variance_ratio_cumulative, marker='.')
plt.xlabel('Número de Componentes Principais')
plt.ylabel('Proporção Cumulativa da Variância Explicada')
plt.title('Proporção Cumulativa da Variância Explicada pelo Número de Componentes Principais')
plt.show()

Não há um valor "ideal"  para a proporção da variância explicada ao escolher o número de componentes principais em uma técnica de redução de dimensionalidade como o TruncatedSVD ou PCA. A escolha depende do contexto específico do problema e dos requisitos da  aplicação.

No entanto,é possível considerar algumas orientações gerais que podem ajudar na tomada de decisão:

1. **80-90% de Variância Explicada:**
   - Geralmente, um bom ponto de partida é selecionar um número de componentes principais que explique entre 80% e 90% da variância original. Isso pode ser suficiente para reter a maioria das informações relevantes enquanto ainda realiza uma redução significativa na dimensionalidade.

2. **Analisar o Gráfico da Proporção Cumulativa:**
   - Analisar o gráfico da proporção cumulativa da variância explicada em relação ao número de componentes principais. Busca-se o ponto em que a curva começa a nivelar, pois isso indica uma diminuição na taxa de ganho de informação.

3. **Avaliação do Desempenho do Modelo:**
   - Avaliar o desempenho do  modelo com diferentes números de componentes principais para determinar se um número menor ou maior de componentes principais é mais benéfico.

4. **Requisitos Computacionais:**
   - Considerar os requisitos computacionais. Reduzir significativamente a dimensionalidade pode acelerar o treinamento do modelo, mas pode não ser necessário para todos os casos.

In [ ]:
model1=Pipeline(steps = [('processing',processing),("pca",TruncatedSVD(n_components=720,random_state=0)),('modeling',LinearRegression())])
model1.fit(X,y)

In [ ]:
model2=Pipeline(steps = [('processing',processing),("pca",TruncatedSVD(n_components=720,random_state=0)),('modeling',AdaBoostRegressor())])
model2.fit(X,y)

In [ ]:
model3=Pipeline(steps = [('processing',processing),("pca",TruncatedSVD(n_components=720,random_state=0)),('modeling',GradientBoostingRegressor())])
model3.fit(X,y)

#Scores

In [ ]:
print(model0.score(X,y))
print(model1.score(X,y))
print(model2.score(X,y))
print(model3.score(X,y))


O método score em modelos de regressão, calcula o coeficiente de determinação R² do modelo. O R² é uma métrica que indica a proporção da variabilidade na variável dependente que é explicada pelo modelo

O R² varia de 0 a 1, e um valor mais próximo de 1 indica que o modelo é capaz de explicar uma maior proporção da variabilidade na variável dependente.

No entanto, é importante notar que o R² não é uma métrica infalível e pode não ser suficiente por si só para avaliar completamente a adequação de um modelo. Outras métricas, como o erro médio absoluto (MAE) ou o erro médio quadrático (MSE), também são úteis para avaliar diferentes aspectos do desempenho do modelo.

Ao utilizar o R², é sempre recomendável considerar outras métricas e realizar uma análise abrangente do desempenho do modelo.

1. **Erro Médio Absoluto (MAE):**
   - O MAE é a média das diferenças absolutas entre as previsões do modelo e os valores reais.
   - O MAE fornece uma medida da magnitude média dos erros, independentemente da direção.

2. **Erro Médio Quadrático (MSE):**
   - O MSE é a média dos quadrados das diferenças entre as previsões do modelo e os valores reais.
   - O MSE penaliza erros maiores devido ao termo quadrático.

3. **Raiz Quadrada do Erro Médio Quadrático (RMSE):**
   - O RMSE é a raiz quadrada do MSE e fornece uma interpretação na mesma unidade da variável dependente.

4. **Erro Percentual Absoluto Médio (MAPE):**
   - O MAPE é uma métrica de erro percentual que expressa o erro como uma porcentagem da média dos valores reais. É calculado como:
   - É útil quando se deseja avaliar o erro relativo em termos percentuais.

5. Coeficiente de Correlação de Pearson:

  - Mede a força e a direção da relação linear entre as previsões e os valores reais.
  - Varia de -1 a 1, onde 1 indica uma correlação positiva perfeita, -1 indica uma correlação negativa perfeita e 0 indica ausência de correlação.


In [ ]:
import joblib
joblib.dump(model0, 'carbPred0.pkl')

In [ ]:
loaded_model = joblib.load('/content/carbPred0.pkl')
y_pred = loaded_model.predict(X)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import matthews_corrcoef

# Calculando métricas de regressão
mae = mean_absolute_error(y, y_pred)
mse = mean_squared_error(y, y_pred)
rmse = mean_squared_error(y, y_pred, squared=False)  # Para obter a raiz quadrada do MSE
r2 = r2_score(y, y_pred)

# Calculando métrica de erro percentual
mape = mean_absolute_percentage_error(y, y_pred)

# Calculando coeficiente de correlação de Pearson
correlation = np.corrcoef(y, y_pred)[0, 1]

print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R²:", r2)
print("MAPE:", mape)
print("Coeficiente de Correlação de Pearson:", correlation)

In [ ]:
# Define your pipeline
pipe = Pipeline([
    ("processing", processing),
    ("pca", TruncatedSVD(n_components=720, random_state=0)),
    ("modeling", GradientBoostingRegressor())
])


In [ ]:
from sklearn.model_selection import KFold

# Criar um objeto KFold
kf = KFold(n_splits=10, shuffle=True, random_state=0)

# Treinar o modelo usando validação cruzada
scores = cross_val_score(
    pipe, X, y, scoring="r2", cv=kf
)

# Imprimir as pontuações de avaliação
print(scores)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_response.py", line 218, in _get_response_values
    y_pred, pos_label = estimator.predict(X), None
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 514, in predict
    Xt = transform.transform(Xt)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils